#Cruce Validacion
Este Markdown tiene como objetivo, Cruzar los datos entre los datos de validacion y notas.
Para este objetivo primero importamos ambos dataset y exploramos sus atributos, y ademas imprimimos los valores del dataset notas.

In [2]:
import pandas as pd

validacion = pd.read_csv('logs_validacion.csv')
notas = pd.read_csv('notas.csv')

column_names_df = notas.columns
column_names_validacion = validacion.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'validacion':", column_names_validacion)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)
notas = notas.replace('Not Attempted', 0)
print(notas)


Column names of 'df': Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 2

#Transformacion de datos horas
Realizamos diversas operaciones de ingeniería de características en el DataFrame validacion, centradas en el análisis temporal de las actividades de los usuarios. Las operaciones se dividen en cuatro segmentos principales:

*Transformación de la columna de tiempo:* Convertimos la columna time a un objeto de fecha y hora para extraer información relevante, como el mes, el día y la hora de cada registro.

*Duración de las sesiones:* Definimos una "sesión" como la actividad de un usuario durante un día específico. Calculamos la duración de cada sesión al determinar el tiempo entre el primer y el último registro de actividad en ese día.

*Estadísticos de duración de las sesiones:* Para cada usuario, calculamos estadísticas descriptivas (media y desviación estándar) sobre la duración de sus sesiones. Estas métricas proporcionan una visión general del comportamiento típico de un usuario y su variabilidad.

*Duración en días del uso de EOL:* Calculamos la duración total (en días) de la actividad de un usuario en la plataforma EOL, determinando la diferencia entre su primera y última fecha registrada.

Por ultimo, integramos estas características recién generadas al DataFrame notas, enriqueciendo así el conjunto de datos con información temporal valiosa sobre las interacciones de los usuarios en la plataforma.

In [3]:
#ACÁ SE TOMA INFO DE LOG_TEST PARA SUPLIR LA DATA DE NOTAS

# Convert the 'time_column' to a datetime object
validacion['time'] = pd.to_datetime(validacion['time'])

# Create new columns for month, day, and time
validacion['month'] = validacion['time'].dt.month
validacion['day'] = validacion['time'].dt.day
validacion['hora'] = validacion['time'].dt.strftime('%H:%M:%S')
#----------------------------------

# DURACION DE SESIONES (una sesion es un día con registro de ingreso)

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = validacion.groupby(['username', 'month', 'day'])['time'].agg(['min', 'max']).reset_index()

grouped['time_difference'] = (grouped['max'] - grouped['min']).dt.total_seconds() / 3600

# Merge the 'time_difference' column from the 'grouped' DataFrame into the 'eventos' DataFrame
validacion = validacion.merge(grouped[['username', 'month', 'day', 'time_difference']], on=['username', 'month', 'day'], how='left')

#----------------------------------
#ESTADISTICOS DE DURACION DE SESIONES

# Group the DataFrame by 'username' and calculate the average and standard deviation of 'time_difference'
grouped = validacion.groupby('username')['time_difference'].agg(['mean', 'std']).reset_index()

# Rename the columns for clarity
grouped.columns = ['username', 'average_time_difference', 'std_time_difference']

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'average_time_difference', 'std_time_difference']].drop_duplicates(), on='username', how='left')

#----------------------------------
#DURACION EN DÍAS DE SU USO DE EOL
# Extract the date part
validacion['date'] = validacion['time'].dt.date
# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = validacion.groupby(['username'])['date'].agg(['min', 'max']).reset_index()
grouped['min'] = pd.to_datetime(grouped['min'])
grouped['max'] = pd.to_datetime(grouped['max'])

# Calculate the time difference in hours
grouped['duracionEOL'] = (grouped['max'] - grouped['min']).dt.days

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracionEOL']].drop_duplicates(), on='username', how='left')

#Frecuencia de Sesiones
A continuacion nos enfocamos en calcular la frecuencia de sesiones por mes para cada usuario en el DataFrame validacion.

**Identificación de meses únicos:** Extraemos los meses únicos presentes en el DataFrame validacion para saber qué meses analizar.

*Preparación de datos para almacenar resultados:* Creamos un diccionario result_data que tendrá como clave username y como valores la cantidad de sesiones distintas que un usuario tuvo en cada mes. Una sesión distinta se define como un día específico en el que el usuario tuvo actividad.

*Cálculo de la frecuencia de sesiones por mes:* Para cada mes identificado, calculamos la cantidad de días únicos (sesiones) en los que cada usuario tuvo actividad. Estos valores se agregan al diccionario result_data.

*Conversión a DataFrame:* Convertimos el diccionario result_data a un DataFrame result_df.

*Renombrar columnas:* Renombramos las columnas del DataFrame para que tengan nombres más descriptivos, como 'Num Sesiones de Agosto', 'Num Sesiones de Septiembre', etc.

*Llenado de valores faltantes:* Rellenamos los valores faltantes (NaN) con 0, lo que indica que un usuario no tuvo sesiones en un mes particular.

*Integración con el DataFrame notas:* Combinamos result_df con el DataFrame notas basándonos en la columna username. Esto añade las características de frecuencia de sesiones por mes al DataFrame principal.

Por ultimo, imprimimos el DataFrame notas actualizado para visualizar los cambios realizados.

In [4]:

#----------------------------------
#FRECUENCIA DE SESIONES POR MES


distinct_months = validacion['month'].unique()

# Create a new dataframe to store the results
result_data = {'username': validacion['username']}
for month in distinct_months:
    result_data[month] = validacion['username'].map(
        validacion[validacion['month'] == month].groupby('username')['day'].nunique()
    )

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(result_data)

# Rename multiple columns
result_df.rename(columns={'8': 'Num Sesiones de Agosto', '9': 'Num Sesiones de Septiembre', '10': 'Num Sesiones de Noviembre'}, inplace=True)


result_df.fillna(0, inplace=True)

# Merge the 'result_df' with the 'df_notas' dataframe based on the 'username' column
notas = notas.merge(result_df, on='username', how='left')

# Print the updated 'eventos' DataFrame
print(notas)


        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
2                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
3                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
4                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
...            ...                                                ...    ...   
196820         608  1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...   0.51   
196821         608  1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...   0.51   
196822         608  1e1188e0478a823fc3f411ef6baed0f6abc324d5b70e60...   0.51   
196823         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
196824         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled Quiz 1: Uso de mod

#Obtencion de Valores Unicos
El código proporcionado tiene como objetivo identificar y visualizar los distintos tipos de eventos agrupados que están presentes en el DataFrame validacion. Para ello, se extraen los valores únicos de la columna grouped_event_type y se imprimen para su revisión.

Esto con el fin de validar la Calidad de los Datos, ya que, al visualizar los valores únicos, también podemos detectar posibles inconsistencias o errores en los datos. Por ejemplo, si esperamos ciertos tipos de eventos pero no aparecen en la lista, o si vemos tipos que no reconocemos, esto podría indicar problemas que necesitan ser abordados.

In [5]:
# Get unique values from a specific column
unique_values = validacion['grouped_event_type'].unique()
print(unique_values)

['/courses/(course-code)/course/'
 'edx.bi.course.upgrade.sidebarupsell.displayed'
 '/courses/(course-code)/course_wiki' 'edx.ui.lms.link_clicked'
 '/courses/(course-code)/wiki/(course-code)/'
 '/courses/(course-code)/progress'
 '/courses/(course-code)/jump_to/(block-code)@vertical@(uuid)'
 '/courses/(course-code)/courseware/(uuid1)/(uuid2)/(tab-n)'
 '/courses/(course-code)/xblock/block-v1:(block-code)@html+block@(uuid)/handler/publish_completion'
 '/courses/(course-code)/courseware/(uuid-code1)/(uuid-code2)/(some-path)'
 'edx.ui.lms.sequence.next_selected' 'load_video' 'play_video'
 'pause_video'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/goto_position'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/get_completion'
 'seq_goto' '/courses/(course-code)/discussion/(discussion-id)/'
 '/courses/(course-code)/(uuid-course)/' 'seek_video' 'seq_next'
 'edx.ui.lms.sequence.previous

# Estadisticos de la sesion
La siguiente parte tiene como objetivo extraer estadisticas por cada sesion, por lo que se realizan las siguientes actividades:

Definición de Tipos de Eventos: Se enumeran los tipos de eventos de interés en validacion_types. Estos representan diferentes acciones que un usuario puede realizar en la plataforma.

Conversión a Variables Booleanas:
Para cada tipo de evento en validacion_types, se crea una nueva columna en validacion que tiene un valor de 1 si el evento corresponde a ese tipo y 0 en caso contrario. Esto facilita la suma y el análisis posterior.

Agrupación por Usuario, Mes y Día:
Los datos se agrupan por username, month y day, y se suman las columnas de eventos para obtener la cantidad total de cada tipo de evento que un usuario realizó en una sesión específica.

Cálculo de Estadísticas:
Para cada usuario, se calculan la media y la desviación estándar de la cantidad de veces que realizaron cada tipo de evento en todas sus sesiones. Estas estadísticas proporcionan una visión general del comportamiento típico de un usuario y su variabilidad.

Aplanar las Columnas:
Se modifica la estructura del DataFrame para que tenga un formato más manejable, renombrando las columnas para que reflejen claramente las métricas calculadas.

Integración con el DataFrame notas:
Se combinan las estadísticas calculadas con el DataFrame notas, enriqueciendo así el conjunto de datos con información sobre el comportamiento de los usuarios en la plataforma.

In [6]:
#SE ENCUENTRAN LOS ESTADISTICOS DE LAS ACTIVIDADES QUE REALIZA EN CADA SESION
# IE CANTIDAD DE VECES QUE REALIZA ALGUNA ACTIVIDAD EN UNA SESION (MONTH, DAY)
# List of event types to create columns for
validacion_types = ['page_close', 'problem_graded', 'problem_check', 'problem_show', 'seg_prev',
               'seg_next', 'seg_goto', 'load_video', 'play_video', 'pause_video', 'speed_change_video']

# Iterate through the event types and create columns with boolean values
for validacion_type in validacion_types:
    validacion[validacion_type] = (validacion['grouped_event_type'] == validacion_type).astype(int)

grouped_eventos = validacion.groupby(['username', 'month', 'day'])[validacion_types].sum().reset_index()



# Group the sub_df DataFrame by 'username' and calculate the mean and standard deviation for each event type

agg_df = grouped_eventos.groupby('username').agg({
    'page_close': ['mean', 'std'],
    'problem_graded': ['mean', 'std'],
    'problem_check': ['mean', 'std'],
    'problem_show': ['mean', 'std'],
    'seg_prev': ['mean', 'std'],
    'seg_next': ['mean', 'std'],
    'seg_goto': ['mean', 'std'],
    'load_video': ['mean', 'std'],
    'play_video': ['mean', 'std'],
    'pause_video': ['mean', 'std'],
    'speed_change_video': ['mean', 'std']
}).reset_index()


# Aplanar las columnas de múltiples niveles en 'agg_df'
agg_df.columns = ['_'.join(col).rstrip() for col in agg_df.columns.values]


# Renombrar la columna en 'agg_df'
agg_df = agg_df.rename(columns={'username_': 'username'})

# Combinar el 'agg_df' modificado con el DataFrame 'notas'
notas = notas.merge(agg_df, on='username', how='left')

# Display the resulting 'notas' DataFrame


# Conteo Actividades por Capitulo
Se realiza un conteo de las actividades por capitulo de tal manera de encontrar los capitualos en los que se haya estudiados mas o menos

In [7]:
#COUNT ACTIVIDADES PER CHAPTER (encontrar chapters que haya estudioado mas/menos)

grouped = validacion.groupby(['username', 'chapter'])['grouped_event_type'].count().reset_index()

pivoted_df = grouped.pivot(index='username', columns='chapter', values='grouped_event_type').fillna(0)

# Reset the index to make 'column1' a regular column
pivoted_df.reset_index(inplace=True)

# Rename columns for clarity
pivoted_df.columns.name = None

# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(pivoted_df, on='username', how='left')


In [8]:
#CREAMOS UN DICCIONARIO CON PARA PODER GUARDAR A LOS USUARIOS CON NOMBRES MAS PEQUEÑOS
# Genera nombres de usuario acortados automáticamente
notas['user_id'] = ['u' + str(i+1) for i in range(len(notas))]

# Crea un diccionario para mapear nombres acortados a nombres largos
diccionario_mapeo = dict(zip(notas['user_id'], notas['username']))

notas = notas.drop('username', axis=1)

column_order = ['user_id'] + [col for col in notas if col != 'user_id']
notas = notas[column_order]

In [9]:
notas.sample(50)

,user_id,Unnamed: 0,Grade,Grade Scaled,Quiz 1: Uso de modelos matemáticos en la economía,Quiz 2: Motivación y Función de Producción,Quiz 3: Preferencias y Toma de Decisiones,Quiz 4: Salario y Efectos,Quiz 5: Explicando Diferencias y Conclusión,Quiz 6: Introducción y conceptos iniciales,...,speed_change_video_std,0,1,2,3,4,5,6,7,none_page
100514,u100515,322,0.94,6.6,1.0,1.0,1.0,1.0,1.0,1.0,...,1.807272,7.0,372.0,591.0,757.0,204.0,503.0,561.0,376.0,367.0
107925,u107926,338,0.88,6.1,1.0,1.0,1.0,1.0,1.0,1.0,...,0.799816,23.0,30.0,58.0,46.0,43.0,30.0,109.0,66.0,262.0
25055,u25056,78,0.90,6.2,1.0,1.0,0.0,1.0,0.0,1.0,...,0.000000,32.0,79.0,53.0,93.0,86.0,27.0,85.0,17.0,387.0
49006,u49007,155,0.91,6.3,1.0,1.0,1.0,1.0,1.0,1.0,...,0.784465,5.0,66.0,224.0,521.0,341.0,472.0,471.0,416.0,422.0
13812,u13813,53,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.084403,24.0,206.0,287.0,628.0,487.0,572.0,334.0,135.0,428.0
144391,u144392,424,0.99,6.9,1.0,1.0,1.0,1.0,1.0,1.0,...,6.939921,26.0,1557.0,2025.0,3185.0,118.0,408.0,329.0,57.0,553.0
149763,u149764,424,0.99,6.9,1.0,1.0,1.0,1.0,1.0,1.0,...,6.939921,26.0,1557.0,2025.0,3185.0,118.0,408.0,329.0,57.0,553.0
104602,u104603,334,0.87,6.0,1.0,1.0,1.0,1.0,1.0,0,...,0.235702,12.0,46.0,36.0,98.0,86.0,42.0,72.0,130.0,309.0
111498,u111499,342,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.437591,8.0,90.0,101.0,120.0,76.0,32.0,20.0,15.0,266.0
79465,u79466,248,1.00,7.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.877058,3.0,28.0,37.0,92.0,124.0,22.0,18.0,27.0,252.0


In [10]:
notas.to_csv('data_para_validar.csv')